In [ ]:
import pandas as pd
import os
from google.colab import drive
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from pandas.errors import EmptyDataError

# 1. Autenticação
filename = 'autenticacao' #documento de autenticação do google
scopes = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(filename, scopes)
client = gspread.authorize(credentials)

# Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

def process_and_update_sheet(folder_path, sheet_name, name_replacements, key_cols_specific=None):
    """
    Processa arquivos CSV de uma pasta, limpa os dados e atualiza uma planilha do Google Sheets.

    Args:
        folder_path (str): Caminho da pasta onde os arquivos CSV estão localizados.
        sheet_name (str): Nome da planilha no Google Sheets a ser atualizada.
        name_replacements (dict): Dicionário para substituir os nomes de arquivos por nomes de artistas.
        key_cols_specific (list, optional): Lista de colunas a serem usadas como chaves de unicidade
                                            para este tipo específico de dados.
                                            Se None, a lógica padrão será usada.
    """
    print(f"\n--- Processando e atualizando a planilha: {sheet_name} ---")

    try:
        spreadsheet = client.open(sheet_name)
        worksheet = spreadsheet.worksheet("Sheet1")
        df_existente = get_as_dataframe(worksheet).dropna(how='all')
        print(f"Dados existentes na planilha '{sheet_name}' carregados. Linhas: {len(df_existente)}")

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Aviso: Planilha '{sheet_name}' não encontrada. Criando uma nova.")
        spreadsheet = client.create(sheet_name)
        worksheet = spreadsheet.get_worksheet(0)
        df_existente = pd.DataFrame()

    except Exception as e:
        print(f"Erro ao abrir ou ler a planilha '{sheet_name}': {e}")
        return

    arquivos_csv = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    lista_de_dataframes = []

    for arquivo in arquivos_csv:
        caminho_arquivo = os.path.join(folder_path, arquivo)
        # print(f"Lendo arquivo: {arquivo}") # Removido para um log mais limpo se muitos arquivos

        try:
            if os.path.getsize(caminho_arquivo) == 0:
                print(f"Aviso: o arquivo {arquivo} está vazio e foi ignorado.")
                continue

            df = pd.read_csv(caminho_arquivo)
            if not df.empty:
                df['nome_arquivo_origem'] = arquivo.replace('.csv', '')
                lista_de_dataframes.append(df)
            else:
                print(f"  Aviso: O arquivo '{arquivo}' foi lido, mas resultou em um DataFrame vazio. Ignorado.")

        except EmptyDataError:
            print(f"Aviso: o arquivo {arquivo} está vazio ou sem colunas e foi ignorado.")
        except pd.errors.ParserError as pe:
            print(f"Erro de parsing no arquivo {arquivo}: {pe}. Ignorado.")
        except Exception as e:
            print(f"Erro inesperado ao ler {arquivo}: {e}. Ignorado.")

    if not lista_de_dataframes:
        print(f"Nenhum arquivo CSV válido encontrado na pasta: {folder_path}. Abortando atualização da planilha.")
        return

    # --- Início da correção da UnboundLocalError ---
    # Define as colunas-chave com base nos dados e na especificidade opcional ANTES da lógica de df_existente
    key_columns = []
    if key_cols_specific:
        key_columns = key_cols_specific
    elif 'nome_arquivo_origem' in lista_de_dataframes[0].columns and 'date' in lista_de_dataframes[0].columns:
        key_columns = ['nome_arquivo_origem', 'date']
    elif 'nome_arquivo_origem' in lista_de_dataframes[0].columns and 'date_added' in lista_de_dataframes[0].columns and 'title' in lista_de_dataframes[0].columns:
        key_columns = ['nome_arquivo_origem', 'title', 'date_added']
    elif 'nome_arquivo_origem' in lista_de_dataframes[0].columns and 'release_date' in lista_de_dataframes[0].columns and 'song' in lista_de_dataframes[0].columns:
        key_columns = ['nome_arquivo_origem', 'song', 'release_date']
    elif 'nome_arquivo_origem' in lista_de_dataframes[0].columns:
         key_columns = ['nome_arquivo_origem'] # Fallback
    else:
        print("Erro: Não foi possível determinar as colunas-chave para identificação de duplicatas. Verifique a estrutura dos CSVs e/ou defina 'key_cols_specific'. Abortando.")
        return

    print(f"Chaves de comparação para duplicatas: {key_columns}") # DEBUG

    tabela_nova = pd.concat(lista_de_dataframes, ignore_index=True)

    # Conversão de datas: feita antes da comparação de chaves
    if 'date' in tabela_nova.columns:
        tabela_nova['date'] = tabela_nova['date'].astype(str)
        tabela_nova['date'] = pd.to_datetime(tabela_nova['date'], errors='coerce').dt.date
    if 'release_date' in tabela_nova.columns:
        tabela_nova['release_date'] = tabela_nova['release_date'].astype(str)
        tabela_nova['release_date'] = pd.to_datetime(tabela_nova['release_date'], errors='coerce').dt.date
    if 'date_added' in tabela_nova.columns:
        tabela_nova['date_added'] = tabela_nova['date_added'].astype(str)
        tabela_nova['date_added'] = pd.to_datetime(tabela_nova['date_added'], errors='coerce').dt.date

    # Criação da coluna 'artista'
    if 'nome_arquivo_origem' in tabela_nova.columns:
        tabela_nova['artista'] = tabela_nova['nome_arquivo_origem'].replace(name_replacements)
    else:
        print("Aviso: Coluna 'nome_arquivo_origem' não encontrada na tabela consolidada. Não foi possível renomear artistas. Abortando.")
        return

    # Garante que as colunas chave existam em ambos os DataFrames (para evitar KeyError no df_existente)
    for col in key_columns:
        if col not in tabela_nova.columns:
            print(f"Erro: Coluna '{col}' necessária para a chave de duplicidade não encontrada na tabela_nova. Abortando.")
            return
        if col not in df_existente.columns and not df_existente.empty: # Só verifica se df_existente NÃO está vazia
            print(f"Erro: Coluna '{col}' necessária para a chave de duplicidade não encontrada na planilha existente ('{sheet_name}'). Adicione a coluna manualmente à sua planilha do Google Sheets. Abortando.")
            return

        # Converte as colunas de data para string se forem usadas como chave, para comparação consistente
        if col in ['date', 'date_added', 'release_date']:
            if col in df_existente.columns: # Apenas se a coluna realmente existe em df_existente
                df_existente[col] = df_existente[col].astype(str)
            tabela_nova[col] = tabela_nova[col].astype(str)

    # --- Lógica de identificação de novos dados ---
    if not df_existente.empty:
        existing_keys = set(tuple(row[col] for col in key_columns) for _, row in df_existente[key_columns].iterrows())
        df_novos_dados = tabela_nova[tabela_nova.apply(lambda row: tuple(row[col] for col in key_columns) not in existing_keys, axis=1)]
        print(f"Novas linhas identificadas para adição: {len(df_novos_dados)}")
    else:
        df_novos_dados = tabela_nova
        print(f"Planilha existente vazia. Todas as {len(df_novos_dados)} linhas serão adicionadas como novas.")

    if not df_novos_dados.empty:
        df_atualizado = pd.concat([df_existente, df_novos_dados], ignore_index=True)

        # A remoção de duplicatas é feita agora após a concatenação, usando as key_columns
        df_atualizado.drop_duplicates(subset=key_columns, inplace=True)
        print(f"Duplicatas removidas com base em {key_columns}.")

        # Garante que as colunas de data estejam no formato correto antes de enviar para o Sheets
        if 'date' in df_atualizado.columns:
            df_atualizado['date'] = pd.to_datetime(df_atualizado['date']).dt.strftime('%Y-%m-%d')
        if 'date_added' in df_atualizado.columns:
            df_atualizado['date_added'] = pd.to_datetime(df_atualizado['date_added']).dt.strftime('%Y-%m-%d')
        if 'release_date' in df_atualizado.columns:
            df_atualizado['release_date'] = pd.to_datetime(df_atualizado['release_date']).dt.strftime('%Y-%m-%d')

        worksheet.clear()
        set_with_dataframe(worksheet, df_atualizado)
        print(f"Planilha do Google Sheets '{sheet_name}' atualizada com sucesso com {len(df_novos_dados)} novas linhas.")
    else:
        print(f"Nenhum dado novo para atualizar na planilha '{sheet_name}'.")

# --- Configurações e Chamadas para cada tipo de dado ---

# Audiência
audience_folder = '/content/drive/MyDrive/dashboaddl/audiencia'
audience_replacements = {
    'Marina Melo-audience-timeline': 'Marina Melo',
    'Preta Ary-audience-timeline': 'Preta Ary',
    'Laura Lorenzetti-audience-timeline': 'Laura Lorenzetti',
    'Bruni Chapow-audience-timeline': 'Bruni Chapow',
    'Marissol Mwaba-audience-timeline': 'Marissol Mwaba',
    'Jade Faria-audience-timeline': 'Jade Faria',
    'Joana Castanheira-audience-timeline': 'Joana Castanheira',
    'Rasura-audience-timeline': 'Rasura'
}
process_and_update_sheet(audience_folder, "tabela_consolidada", audience_replacements, key_cols_specific=['nome_arquivo_origem', 'date'])

# Playlist 28 dias
playlist28d_folder = '/content/drive/MyDrive/dashboaddl/p28d'
playlist28d_replacements = {
    'Joana Castanheira-playlists-28day': 'Joana Castanheira',
    'Marina Melo-playlists-28day': 'Marina Melo',
    'Jade Faria-playlists-28day': 'Jade Faria',
    'Laura Lorenzetti-playlists-28day': 'Laura Lorenzetti',
    'Marissol Mwaba-playlists-28day': 'Marissol Mwaba',
    'Preta Ary-playlists-28day': 'Preta Ary',
    'Rasura-playlists-28day': 'Rasura',
    'Bruni Chapow-playlists-28day': 'Bruni Chapow'
}
process_and_update_sheet(playlist28d_folder, "tabela28d", playlist28d_replacements, key_cols_specific=['nome_arquivo_origem', 'title', 'date_added'])

# Playlist 1 ano
playlist1y_folder = '/content/drive/MyDrive/dashboaddl/p1an'
playlist1y_replacements = {
    'Joana Castanheira-playlists-1year': 'Joana Castanheira',
    'Marina Melo-playlists-1year': 'Marina Melo',
    'Marina Melo-playlists-1year (2)': 'Marina Melo',
    'Jade Faria-playlists-1year': 'Jade Faria',
    'Laura Lorenzetti-playlists-1year': 'Laura Lorenzetti',
    'Marissol Mwaba-playlists-1year': 'Marissol Mwaba',
    'Preta Ary-playlists-1year': 'Preta Ary',
    'Rasura-playlists-1year': 'Rasura',
    'Bruni Chapow-playlists-1year': 'Bruni Chapow'
}
process_and_update_sheet(playlist1y_folder, "tabela1an", playlist1y_replacements, key_cols_specific=['nome_arquivo_origem', 'title', 'date_added'])

# Música 1 ano
music1y_folder = '/content/drive/MyDrive/dashboaddl/musi1an'
music1y_replacements = {
    'Joana Castanheira-songs-1year': 'Joana Castanheira',
    'Marina Melo-songs-1year': 'Marina Melo',
    'Jade Faria-songs-1year': 'Jade Faria',
    'Laura Lorenzetti-songs-1year': 'Laura Lorenzetti',
    'Marissol Mwaba-songs-1year': 'Marissol Mwaba',
    'Preta Ary-songs-1year': 'Preta Ary',
    'Rasura-songs-1year': 'Rasura',
    'Bruni Chapow-songs-1year': 'Bruni Chapow'
}
process_and_update_sheet(music1y_folder, "tabelam1an", music1y_replacements, key_cols_specific=['nome_arquivo_origem', 'song', 'release_date'])

# Música 28 dias
music28d_folder = '/content/drive/MyDrive/dashboaddl/musi28d'
music28d_replacements = {
    'Joana Castanheira-songs-28day': 'Joana Castanheira',
    'Marina Melo-songs-28day': 'Marina Melo',
    'Marina Melo-songs-28day (2)': 'Marina Melo',
    'Jade Faria-songs-28day': 'Jade Faria',
    'Laura Lorenzetti-songs-28day': 'Laura Lorenzetti',
    'Marissol Mwaba-songs-28day': 'Marissol Mwaba',
    'Preta Ary-songs-28day': 'Preta Ary',
    'Rasura-songs-28day': 'Rasura',
    'Bruni Chapow-songs-28day': 'Bruni Chapow'
}
process_and_update_sheet(music28d_folder, "tabelam28d_s", music28d_replacements, key_cols_specific=['nome_arquivo_origem', 'song', 'release_date'])